# Advanced Ollama Features Tutorial: 🛠️ Tool Usage 🛠️

In this notebook, we'll explore a powerful feature of Ollama that allows us to equip AI with using custom functions

## 0. Import libraries + Start Ollama server

In [1]:
import subprocess
import os
import socket
import time
import psutil
from ollama import ChatResponse, chat

In [2]:
### Helper functions

def is_port_in_use(port: int = 11434) -> bool:
    """Check if the Ollama port is already in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

def is_ollama_running() -> bool:
    """Check if Ollama process is running"""
    for proc in psutil.process_iter(['name']):
        try:
            if proc.info['name'] == 'ollama':
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False

def ensure_ollama_server():
    """
    Ensures that an Ollama server is running.
    Returns the process object if a new server was started, None if server was already running.
    """
    # First check if the server is already running
    if is_port_in_use() or is_ollama_running():
        print("✅ Ollama server is already running!")
        return None
    
    try:
        # Start the Ollama server
        print("🚀 Starting Ollama server...")
        proc = subprocess.Popen(
            ["ollama", "serve"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            start_new_session=True
        )
        
        # Wait for the server to start (max 10 seconds)
        for _ in range(10):
            if is_port_in_use():
                print(f"✅ Ollama server started successfully (PID: {proc.pid})")
                return proc
            time.sleep(1)
            
        raise TimeoutError("Server didn't start within 10 seconds")
        
    except Exception as e:
        print(f"❌ Error starting Ollama server: {e}")
        if 'proc' in locals():
            proc.terminate()
        raise

#### Start the ollama server and list ollama models

In [3]:
# Start the server if it's not already running
ollama_process = ensure_ollama_server()

🚀 Starting Ollama server...
✅ Ollama server started successfully (PID: 2268460)


In [4]:
!ollama ls
# If you need to pull models, uncomment the following:
# !ollama pull phi4

NAME                      ID              SIZE      MODIFIED   
llama3.2-vision:latest    6f2f9757ae97    7.8 GB    3 days ago    
llama3.1:latest           46e0c10c039e    4.9 GB    3 days ago    
llama3.2:1b               baf6a787fdff    1.3 GB    4 days ago    
llava:latest              8dd30f6b0cb1    4.7 GB    5 days ago    
qwen3:0.6b                7df6b6e09427    522 MB    5 days ago    


## 1. Tool Use: Making AI Use Custom Functions 🛠️

One of the coolest features of advanced AI models is their ability to use tools - custom functions that we define. This allows the AI to perform actual calculations or actions.

In this example, we'll create two simple math functions and let the AI use them to solve problems. Here's how it works:

1. We define some functions (tools) that the AI can use
2. We tell the AI about these tools
3. The AI decides when and how to use them

Let's try it out! We will first create two tools we want the LLM to use rather than think about: addition and substraction

In [13]:
def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers
  Args:
    a (int): The first number
    b (int): The second number
  Returns:
    int: The sum of the two numbers
  """
  # The cast is necessary as returned tool call arguments don't always conform exactly to schema
  # E.g. this would prevent "what is 30 + 12" to produce '3012' instead of 42
  return int(a) + int(b)

def subtract_two_numbers(a: int, b: int) -> int:
  """
  Subtract two numbers
  """
  # The cast is necessary as returned tool call arguments don't always conform exactly to schema
  return int(a) - int(b)

In [14]:
# We now will create a dictionary that has the two functions we implemented
# This is to map tool names → Python callables
available_functions = {
  'add_two_numbers': add_two_numbers,
  'subtract_two_numbers': subtract_two_numbers,
}

### This step is optional but good practice to implement.

Tools can be manually defined by schema's which can be passed into chat (i.e the context of the LLM). This allows the LLM to have more knowledge on a tool's description, input arguments, return arguments, etc.

For now we will only create a schema for the substract tool to show that this is optional and not mandatory when using functions

In [15]:
# Tools can still be manually defined and passed into chat
subtract_two_numbers_tool = {
  'type': 'function',
  'function': {
    'name': 'subtract_two_numbers',                 # Has to match exactly the syntax for the actual python subtract_two_numbers function
    'description': 'Subtract two numbers',
    'parameters': {
      'type': 'object',
      'required': ['a', 'b'],
      'properties': {
        'a': {'type': 'integer', 'description': 'The first number'},
        'b': {'type': 'integer', 'description': 'The second number'},
      },
    },
  },
}

#### Let's now come up with a good prompt in which the LLM will be using the addition/substraction tool

In [16]:
messages = [
    {
        'role': 'user', 
        'content': 'What is three plus one?'     # Prompt to give to the LLM
    }
]
print('Prompt:', messages[0]['content'])

Prompt: What is three plus one?


#### Now we can run the LLM to get a response but we include a list of the 2 available tools we implemented.

Note that we use the actual python function for add_two_numbers whereas the schema for subtract_two_numbers_tool. This is done to show you the 2 ways to do tool usage with LLM's in ollama.

In [20]:
response: ChatResponse = chat(
  'llama3.1',
  messages=messages,
  tools=[add_two_numbers, subtract_two_numbers_tool],  # Note: we use the actual python function for addition but the schema for subtract_two_numbers_tool
)

In [21]:
if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')

  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  final_response = chat('llama3.1', messages=messages)
  print('Final response:', final_response.message.content)
else:
  print('No tool calls returned from model')

Calling function: add_two_numbers
Arguments: {'a': 3, 'b': 1}
Function output: 4
Final response: The answer to the equation 3 + 1 is 4.


### Now let's try a prompt in which the LLM will try to use both tools (i.e both addition and substraction)

In [24]:
messages = [
    {
        'role': 'user', 
        'content': 'What is three plus one? What is four minus two'     # Prompt to give to the LLM
    }
]
print('Prompt:', messages[0]['content'])

Prompt: What is three plus one? What is four minus two


In [25]:
response: ChatResponse = chat(
  'llama3.1',
  messages=messages,
  tools=[add_two_numbers, subtract_two_numbers_tool],  # Note: we use the actual python function for addition but the schema for subtract_two_numbers_tool
)

In [26]:
if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')

  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  final_response = chat('llama3.1', messages=messages)
  print('Final response:', final_response.message.content)
else:
  print('No tool calls returned from model')

Calling function: add_two_numbers
Arguments: {'a': 3, 'b': 1}
Function output: 4
Calling function: subtract_two_numbers
Arguments: {'a': 4, 'b': 2}
Function output: 2
Final response: The answer to the first question is 4. The answer to the second question is 2.


## Exercise 1: Conversion of Fahrenheit to Celsius -- with no Schema

Now it is your turn to try and implement tools for other tasks.
The first tools you have to implement are tools to convert Fahrenheit to Celsius (and vice-versa)

In [32]:
def fahrenheit_to_celsius(fahrenheit: float) -> float:
  """
  Convert Fahrenheit to Celsius.

  Args:
    fahrenheit (float): Temperature in Fahrenheit.

  Returns:
    float: Temperature in Celsius.
  """
  # Cast to float in case the model gives back a string
  return (float(fahrenheit) - 32.0) * 5.0 / 9.0


def celsius_to_fahrenheit(celsius: float) -> float:
  """
  Convert Celsius to Fahrenheit.

  Args:
    celsius (float): Temperature in Celsius.

  Returns:
    float: Temperature in Fahrenheit.
  """
  return float(celsius) * 9.0 / 5.0 + 32.0

# Map tool names → Python callables
available_functions = {
  "fahrenheit_to_celsius": fahrenheit_to_celsius,
  "celsius_to_fahrenheit": celsius_to_fahrenheit,
}

In [33]:
messages = [
  {
    "role": "user",
    "content": "It's 75°F outside. What is that in Celsius?"
  }
]

print("Prompt:", messages[0]["content"])

Prompt: It's 75°F outside. What is that in Celsius?


In [34]:
response: ChatResponse = chat(
  "llama3.1",
  messages=messages,
  tools=[fahrenheit_to_celsius, celsius_to_fahrenheit],
)

In [35]:
if response.message.tool_calls:
  for tool in response.message.tool_calls:
    if function_to_call := available_functions.get(tool.function.name):
      print("Calling function:", tool.function.name)
      print("Arguments received:", tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print("Function output:", output)
    else:
      print("Function", tool.function.name, "not found")

  # Send the tool output(s) back to the LLM for a natural-language answer
  messages.append(response.message)                            # original assistant reply
  messages.append({                                            # synthetic tool message
      "role": "tool",
      "name": tool.function.name,
      "content": str(output)
  })
  final_response = chat("llama3.1", messages=messages)
  print("Final response:", final_response.message.content)

else:
  print("No tool calls returned from model")

Calling function: fahrenheit_to_celsius
Arguments received: {'fahrenheit': '75'}
Function output: 23.88888888888889
Final response: To convert Fahrenheit to Celsius, you can use the formula:

Celsius = (Fahrenheit - 32) × 5/9

Plugging in 75 for Fahrenheit, we get:

Celsius = (75 - 32) × 5/9
= 43.88888888888889


## Exercise 2: Conversion of Fahrenheit to Celsius -- with a Schema

Now it is your turn to try and implement tools for other tasks.
The first tools you have to implement are tools to convert Fahrenheit to Celsius (and vice-versa)

In [39]:
# ----------------------------------------------------------------------
# Manual JSON-schema description for one of the tools (The other can be passed to `chat` directly, as in the original.)
# ----------------------------------------------------------------------
celsius_to_fahrenheit_tool = {
  "type": "function",
  "function": {
    "name": "celsius_to_fahrenheit",
    "description": "Convert Celsius to Fahrenheit",
    "parameters": {
      "type": "object",
      "required": ["celsius"],
      "properties": {
        "celsius": {
          "type": "number",
          "description": "Temperature in Celsius"
        }
      }
    }
  }
}

In [40]:
messages = [
  {
    "role": "user",
    "content": "It's 75°F outside. What is that in Celsius?"
  }
]

print("Prompt:", messages[0]["content"])

Prompt: It's 75°F outside. What is that in Celsius?


In [41]:
response: ChatResponse = chat(
  "llama3.1",
  messages=messages,
  tools=[fahrenheit_to_celsius, celsius_to_fahrenheit_tool],
)

In [42]:
# ----------------------------------------------------------------------
# If the model used a tool, run it locally and feed the result back
# ----------------------------------------------------------------------
if response.message.tool_calls:
  for tool in response.message.tool_calls:
    if function_to_call := available_functions.get(tool.function.name):
      print("Calling function:", tool.function.name)
      print("Arguments received:", tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print("Function output:", output)
    else:
      print("Function", tool.function.name, "not found")

  # Send the tool output(s) back to the LLM for a natural-language answer
  messages.append(response.message)                            # original assistant reply
  messages.append({                                            # synthetic tool message
      "role": "tool",
      "name": tool.function.name,
      "content": str(output)
  })
  final_response = chat("llama3.1", messages=messages)
  print("Final response:", final_response.message.content)

else:
  print("No tool calls returned from model")

Calling function: fahrenheit_to_celsius
Arguments received: {'fahrenheit': 75}
Function output: 23.88888888888889
Final response: To convert 75°F to Celsius, we subtract 32 from the Fahrenheit temperature and then divide by 1.8.

75 - 32 = 43
43 / 1.8 = 23.889


## Exercise 3: Converting strings to Upper and Lower case
You are now tasked with implementing tools for converting strings in text to Lower and Upper case (and vice-versa).
I.e if the input text is "hello world" then the upper case version of this text is "HELLO WORLD"
I.e if the input text is "Hello World" then the lower case version of this text is "hello world"

In [43]:
def capitalize_words(text: str) -> str:
  """
  Capitalize every word in a sentence.

  Args:
    text (str): The input sentence.

  Returns:
    str: The sentence with each word capitalized.
  """
  # Simple split-and-join to capitalize each word
  return " ".join(word.capitalize() for word in text.split())


def lowercase_words(text: str) -> str:
  """
  Convert every character in the sentence to lowercase.
  """
  return text.lower()

available_functions = {
  'capitalize_words': capitalize_words,
  'lowercase_words': lowercase_words,
}

In [44]:
messages = [{
  'role': 'user',
  'content': 'Please capitalize every word: hello world from ollama',
}]
print('Prompt:', messages[0]['content'])


Prompt: Please capitalize every word: hello world from ollama


In [45]:
response: ChatResponse = chat(
  'llama3.1',
  messages=messages,
  tools=[capitalize_words, lowercase_words],
)

In [46]:
if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')
  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({
    'role': 'tool',
    'content': str(output),
    'name': tool.function.name,
  })

  # Get final response from model with function outputs
  final_response = chat('llama3.1', messages=messages)
  print('Final response:', final_response.message.content)

else:
  print('No tool calls returned from model')

Calling function: capitalize_words
Arguments: {'text': 'hello world from ollama'}
Function output: Hello World From Ollama
Final response: Here is the corrected response with each word capitalized:

Hello World From Ollama


## Exercise 4: More Math tools


In [47]:
def power_two_numbers(a: float, b: float) -> float:
  """
  Raise one number to the power of another

  Args:
    a (float): The base
    b (float): The exponent

  Returns:
    float: a raised to the power b
  """
  # Casts ensure the operation is numeric even if the model returns strings
  return float(a) ** float(b)

def safe_divide(a: float, b: float) -> float:
  """
  Divide two numbers safely (∞ if dividing by zero)

  Args:
    a (float): The numerator
    b (float): The denominator

  Returns:
    float: a / b, or ∞ if b == 0
  """
  try:
    return float(a) / float(b)
  except ZeroDivisionError:
    return float('inf')

# Map of callable tools
available_functions = {
  'power_two_numbers': power_two_numbers,
  'safe_divide': safe_divide,
}

In [48]:
# Initial user query
messages = [{'role': 'user', 'content': 'What is five to the power of three?'}]
print('Prompt:', messages[0]['content'])

Prompt: What is five to the power of three?


In [49]:
# Call the model with both tools: one passed as a callable, one as a dict
response: ChatResponse = chat(
  'llama3.1',
  messages=messages,
  tools=[power_two_numbers, safe_divide],
)

In [51]:
if response.message.tool_calls:
  for tool in response.message.tool_calls:
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')

  # Provide the tool output back to the model for a natural-language answer
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})
  final_response = chat('llama3.1', messages=messages)
  print('Final response:', final_response.message.content)
else:
  print('No tool calls returned from model')

Calling function: power_two_numbers
Arguments: {'a': 5, 'b': 3}
Function output: 125.0
Final response: To calculate this, we multiply 5 by itself three times:

5 × 5 = 25
25 × 5 = 125

So, five to the power of three is 125.
